In [148]:
import pandas as pd



pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

referendum_data = pd.read_csv(
    'data/EU-referendum-result-data.csv'
)

# ages = pd.read_csv(
#     'data/ages.csv'
# )
density = pd.read_csv(
    'data/density.csv'
)
# ethnic_groups = pd.read_csv(
#     'data/ethnic_groups.csv'
# )
health = pd.read_csv(
    'data/health.csv'
)
sex = pd.read_csv(
    'data/sex.csv'
)

areas_lat_lon = pd.read_csv(
    'data/areas_lat_lon.csv'
)


# health['Very good health'] = health['Very good health'].apply(lambda x: x.replace(",", ""))
# health['Good health'] = health['Good health'].apply(lambda x: x.replace(",", ""))
# health['Fair health'] = health['Fair health'].apply(lambda x: x.replace(",", ""))
# health['Very bad health'] = health['Very bad health'].apply(lambda x: x.replace(",", ""))
# health['Bad health'] = health['Bad health'].apply(lambda x: x.replace(",", ""))
# health['Area code']

# sex['Males'] = sex['Males'].apply(lambda x: x.replace(",", ""))
# sex['Females'] = sex['Females'].apply(lambda x: x.replace(",", ""))
# sex['Area code']


# density['Density']
# density['Area code']


health = health[
    ['Very good health',
    'Good health',
    'Fair health',
    'Very bad health',
    'Bad health',
    'Area code',]
]
sex = sex[['Males', 'Females', 'Area code']]
density = density[['Area code', 'Density']]
referendum_data = referendum_data[[
    'Area_Code', 'Area', 'Remain', 'Leave'
]]
referendum_data = referendum_data.rename(columns={'Area_Code': 'Area code'})
areas_lat_lon = areas_lat_lon[[
    'lad17cd', 'lat', 'long'
]]
areas_lat_lon=areas_lat_lon.rename(columns={'lad17cd': 'Area code'})


ward_to_district = pd.read_csv('data/ward_to_local_district.csv')
ward_to_district=ward_to_district.rename(columns={'WD18CD': 'ward_id', 'LAD18CD': 'Area code'})
ward_to_district=ward_to_district[['ward_id', 'Area code']]
# LAD18CD
# WD18CD

ward_codes = pd.read_csv('data/ward-codes-w-30-45.csv')
ward_codes=ward_codes.rename(columns={'ward': 'ward_id'})
ward_codes = ward_codes.merge(
    ward_to_district, on='ward_id', how='left'
)
# ward_codes=ward_codes[ward_codes['Area code'].notnull()]
# ward_codes.groupby("Area code").mean().sort_values("30-45w", ascending=False)
ward_codes['number'] = ward_codes['30-45w'] * ward_codes['pop'] 

ward_codes = ward_codes.drop_duplicates()
ward_result = ward_codes.groupby("Area code").sum().sort_values("number", ascending=False).reset_index()
ward_result = ward_result[['Area code', 'number']]
ward_result.head()
ward_codes.head()

ward_to_district_2 = pd.read_csv('data/ward_to_local_district.csv')
ward_to_district_2
ward_codes = pd.read_csv('data/ward-codes-w-30-45.csv')
ward_codes.sum()

# ward_codes = pd.read_csv('data/ward-codes-w-30-45.csv')
# ward_codes=ward_codes.rename(columns={'ward': 'ward_id'})
# ward_codes = ward_codes.merge(
#     ward_to_district, on='ward_id', how='left'
# )
referendum_data = referendum_data.merge(
    density, on='Area code', how='left'
)
referendum_data = referendum_data.merge(
    sex, on='Area code', how='left'
)

referendum_data = referendum_data.merge(
    health, on='Area code', how='left'
)

referendum_data = referendum_data.merge(
    areas_lat_lon, on='Area code', how='left'
)

def _clean(x):
    try:
        return int(x.replace(",", ""))
    except:
        return x

# referendum_data.to_csv("the_real_data.csv")
# referendum_data['Area code']
referendum_data[referendum_data['Area code'] == 'E07000220']
res = referendum_data.merge(ward_result, on="Area code", how="left")

res['remain_pct'] = res['Remain'] / (res['Remain'] + res['Leave'])
res['women_voters_pct'] = res['Females'].apply(_clean) / (res['Females'].apply(_clean) + res['Males'].apply(_clean))

# .apply(_clean)

res['Very good health'] = res['Very good health'].apply(_clean)
res['Good health'] = res['Good health'].apply(_clean)
res['Fair health'] = res['Fair health'].apply(_clean)
res['Bad health'] = res['Bad health'].apply(_clean)
res['Very bad health'] = res['Very bad health'].apply(_clean)

res['very_good_health_pct'] = res['Very good health'] / (res['Very good health'] + res['Good health'] +res['Bad health'] + res['Fair health'] + res['Very bad health'])
res['very_bad_health_pct'] =  res['Very bad health'] / (res['Very good health'] + res['Good health'] + res['Bad health']+res['Fair health'] + res['Very bad health'])

res = res.sort_values("number", ascending=False)
res





Area code                          Area  Remain   Leave  Density  \
280  E06000052                      Cornwall  140540  182665      1.5   
282  E06000054                     Wiltshire  137258  151637      1.4   
125  E06000047                 County Durham  113521  153877      2.3   
322  W06000015                       Cardiff  101788   67816     24.7   
136  E06000049                 Cheshire East  107962  113163      3.2   
211  E06000042                 Milton Keynes   63393   67063      8.1   
5    E06000056          Central Bedfordshire   69670   89134      3.6   
362  E06000011      East Riding of Yorkshire   78779  120136      1.4   
126  E06000057                Northumberland   82022   96699      NaN   
377  E08000032                      Bradford  104575  123913     14.3   
334  E06000051                    Shropshire   78987  104166      1.0   
274  E06000025         South Gloucestershire   74928   83405      5.3   
279  E06000030                       Swindon   51220   61745      9.1   
323  W06000016             Rhondda Cynon Taf   53973   62590      5.5   
318  W06000011                       Swansea   58307   61936      6.3   
216  E07000004                Aylesbury Vale   52877   53956      1.9   
137  E06000050     Cheshire West and Chester   95455   98082      3.6   
273  E06000024                North Somerset   59572   64976      5.4   
374  E08000017                     Doncaster   46922  104260      5.3   
210  E06000041                     Wokingham   55272   42229      8.6   
324  W06000018                    Caerphilly   39178   53295      6.4   
332  E06000020            Telford and Wrekin   32954   56649      5.7   
331  E06000019      Herefordshire, County of   44148   64122      0.8   
317  W06000010               Carmarthenshire   47654   55381      0.8   
206  E06000037                West Berkshire   48300   44977      2.2   
45   E07000242            East Hertfordshire   42372   42994      NaN   
269  E07000228                    Mid Sussex   46471   41057      4.2   
375  E08000018                     Rotherham   44115   93272      9.0   
133  E06000007                    Warrington   52657   62487     11.2   
328  W06000022                       Newport   32413   41236      7.6   
381  E08000036                     Wakefield   58877  116165      9.6   
313  W06000005                    Flintshire   37867   48930      3.5   
232  E07000091                    New Forest   47199   64541      2.3   
248  E07000177                      Cherwell   40668   41168      2.4   
205  E06000036              Bracknell Forest   29888   35002     10.3   
305  E07000189                South Somerset   42527   56940      1.7   
320  W06000013                      Bridgend   33723   40622      5.6   
250  E07000179             South Oxfordshire   46245   37865      2.0   
319  W06000012             Neath Port Talbot   32651   43001      3.2   
17   E07000072                 Epping Forest   28676   48176      3.7   
224  E07000065                       Wealden   44084   52808      1.8   
251  E07000180           Vale of White Horse   43462   33192      2.1   
265  E07000224                          Arun   34193   56936      6.8   
347  E07000222                       Warwick   47976   33642      4.9   
314  W06000006                       Wrexham   28822   41544      2.7   
35   E07000149                 South Norfolk   38817   41541      1.4   
321  W06000014             Vale of Glamorgan   36681   35628      3.8   
262  E07000216                      Waverley   44341   31601      3.5   
234  E07000093                   Test Valley   36170   39091      1.9   
29   E07000143                     Breckland   26313   47235      1.0   
51   E07000032                  Amber Valley   29319   44501      4.6   
60   E07000130                     Charnwood   43500   50672      6.0   
242  E07000111                     Sevenoaks   32091   38258      3.1   
379  E08000034                      Kirklees   98485  118755   

In [150]:
import numpy as np
res = res.sort_values("remain_pct", ascending=False)
_res = []
for _, row in res.iterrows(): 
    if np.isnan(row['remain_pct']) or np.isnan(row['number']):
        pass
    else:
        x = [row['number'], row['remain_pct'] * 100]
#         print(x)
        _res.append(x)

# xx=pd.DataFrame(_res)
# xx.head()
_res[3]

[9222.0, 62.70808028226039]